# Plotting ADCP Data

This notebook was adapted from Dr. Nick Beird's integrated ocean observing class with his permission. Thanks, Nick!

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} xarray

**We're adding a new package to our repertoire ... xarray!!** 
<br>
Xarray is a package beloved by many for dealing with three+ dimensional data. It is similar to pandas in that it organizes our data in a neat and efficient way. While `pandas` is great for "tabular" data (think, something that you could put in an excel sheet) `xarray` is great for gridded data and building multi-dimensional data "cubes". Nerd-out on xarray facts here: https://docs.xarray.dev/en/stable/

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs #coordinate reference system
import cartopy.feature as cfeature

In [ ]:
ds = xr.open_dataset('os150bb.nc').transpose()
#.transpose will switch the x and y coordinates, like transposing an array
ds

Notice how our dataset `ds` has both dimensions and coordinates. These will come in handy! Lets make a plot to visualize the data we have here. We can use dot indexing to index variables within our dataset. Using the `.plot` function on our xarray dataset, we can plot the `u` variable (east/west velocity) in just a few lines of code. The default `.plot` will plot the indicated variable against the dimensions of the dataset. It will even label your axes for you!

In [ ]:
ds.u.plot(cmap='coolwarm', vmin = -1.5, vmax= 1.5)
ax = plt.gca()
ax.set_facecolor('k')
plt.ylim([25,0])

Notice that the colorbar is diverging becuase we have postive and negative values of `u`. Cannonically, postiive u is pointing eastward and negative u is pointing westward. <br>
Cool! But where in the world are we? Let's make an aerial track plot to find out.

In [ ]:
# 'Mercator' represents lat and lon on a more realistic globe
proj = ccrs.Mercator()
# PlateCarree is the projection that your xarray is currently on
data_crs = ccrs.PlateCarree()
# create plot with the 'Mercator' projection
ax = plt.subplot(projection = proj)

plt.plot( ds.lon, ds.lat, transform = data_crs ) # need to "transform" data from PlateCaree to Mercator
plt.xticks(rotation=45)

ax.coastlines()
ax.add_feature( cfeature.LAND, color='green' )

ax.gridlines(draw_labels=True)

ax.set_extent([-180, -150, 50, 75])

The track goes back and forth over the same transect (see those two deep canyons on our first plot?) <br>
We are going to select for just the first transect by indexing our xarray by time.

In [ ]:
transect1 = ds.sel(time = slice('2017-08-16','2017-08-17 12:00:00'))
transect1

We still have 30 depth cells in our dimensions list, but now we only have 432 timestamps.

In [ ]:
plt.pcolor(transect1.time, transect1.depth[:,1], transect1.v, vmin=-.75, vmax=.75, cmap= 'coolwarm')
plt.colorbar()

plt.ylim([120,0])
plt.xticks(rotation = 45)

Now, we are going to slice our dataset in the other dimension to get just the top depth cell (the surface) and plot a vector map of surface velocities across this transect.

In [ ]:
transect1.depth_cell # checking to see how the depth cells are indexed

In [ ]:
surface_t1 = transect1.sel(depth_cell = 0) # select for the top bin of depth ##NOTE: you can only use '.sel' on ds dimensions
surface_t1.depth.values # make sure we chose the right bin (i.e. that 0 = surface and not 0 = bottom)

In [ ]:
surface_t1 # surface is now a dataset with one dimension

Now that we have our indexed dataset, let's make a quiver plot. A quiver plot will convert the u and v components into a vector with magnitude and direction.

In [ ]:
plt.quiver(surface_t1.lon, surface_t1.lat, surface_t1.u, surface_t1.v)

This plot looks a little busy, so I'm going to plot only every 10th datapoint.

In [ ]:
fig = plt.figure(figsize=(10.0, 10.0))

plt.quiver(surface_t1.lon[::10], surface_t1.lat[::10], surface_t1.u[::10], surface_t1.v[::10])

# Now it's your turn to make some plots

Plot a second transect over the same bathymetric features. <br>
Your second transect should be between '2017-08-09 16:00:00','2017-08-10 18:00:00' <br>
<br>
make the following plots: <br>
(1) cruise track of the second transect in relation to nearby landmasses <br>
(2) cross-section of the v component velocity <br>
(3) a quiver plot of the surface velocities of the second transect
<br>
<br>
How do the two transects differ?